In [1]:
from itertools import product
import numpy as np
from pulp import LpMinimize, LpMaximize, LpProblem
from funcs import make_partition_probabilities, simulate_participants, get_state_probabilities, \
                  set_up_variables, solve, itt_ate, q_ate, true_ate, symbolic_solve

In [2]:
compliance_partitions = ['always_taker', 'complier', 'defier', 'never_taker']
response_partitions = ['always_better', 'helped', 'hurt', 'never_better']
partition_types = np.array(list(product(compliance_partitions, response_partitions)))

In [3]:
partition_types

array([['always_taker', 'always_better'],
       ['always_taker', 'helped'],
       ['always_taker', 'hurt'],
       ['always_taker', 'never_better'],
       ['complier', 'always_better'],
       ['complier', 'helped'],
       ['complier', 'hurt'],
       ['complier', 'never_better'],
       ['defier', 'always_better'],
       ['defier', 'helped'],
       ['defier', 'hurt'],
       ['defier', 'never_better'],
       ['never_taker', 'always_better'],
       ['never_taker', 'helped'],
       ['never_taker', 'hurt'],
       ['never_taker', 'never_better']], dtype='<U13')

In [4]:
def experiment(distribution_floats, n_participants, partition_types):
    partition_p = make_partition_probabilities(distribution_floats)
    df = simulate_participants(partition_p, n_participants, partition_types)
    
    itt_effect = itt_ate(df)
    
    state_p = get_state_probabilities(df)
    q = set_up_variables(partition_types)
    minimize = LpProblem('min', LpMinimize)
    maximize = LpProblem('max', LpMaximize)
    
    try:
        q_min = solve(minimize, q, state_p)
        q_max = solve(maximize, q, state_p)
    except ValueError:
        q_min = {k:np.nan for k,v in q.items()}
        q_max = {k:np.nan for k,v in q.items()}
    
    lower_ate = q_ate(q_min)
    upper_ate = q_ate(q_max)
    
    symbolic_lower_ate, symbolic_upper_ate = symbolic_solve(state_p)
    
    true = true_ate(df)
    
    return true, itt_effect, lower_ate, upper_ate, symbolic_lower_ate, symbolic_upper_ate

In [21]:
distribution_magnitude = [0, 10, 0, 0, 0, 10, 0, 0, 0, 10, 0, 0, 0, 10, 0, 0]
n_participants = 20000

In [22]:
experiment(distribution_magnitude, n_participants, partition_types)

(1.0, 0.0010999999999999899, 0.0011, 1.0, 0.001100000000000101, 1.0)

In [20]:
distribution_magnitude = [0, 0, 0, 0, 0, 10, 4, 0, 0, 10, 4, 0, 0, 10, 4, 0]
experiment(distribution_magnitude, n_participants, partition_types)

(0.4276, 0.0030000000000000027, nan, nan, -0.0827, 0.9048999999999999)

In [19]:
distribution_magnitude = [0, 0, 0, 0, 2, 10, 4, 5, 2, 10, 4, 5, 2, 10, 4, 5]
experiment(distribution_magnitude, n_participants, partition_types)

(0.27825, -0.0012999999999999956, -0.2021, 0.7893, -0.2029, 0.7893)

In [18]:
distribution_magnitude = [0, 0, 0, 0, 2, 15, 4, 8, 2, 15, 4, 8, 2, 15, 4, 8]
experiment(distribution_magnitude, n_participants, partition_types)

(0.38144999999999996,
 -0.005700000000000011,
 -0.1754,
 0.8150999999999999,
 -0.1754,
 0.8150999999999999)

In [17]:
distribution_magnitude = [0, 0, 0, 0, 10, 15, 4, 1, 2, 15, 4, 8, 2, 15, 4, 8]
experiment(distribution_magnitude, n_participants, partition_types)

(0.36249999999999993, 0.003300000000000025, nan, nan, -0.1065, 0.7201)

In [29]:
distribution_magnitude = [0, 37, 4, 5, 10, 38, 4, 1, 2, 15, 4, 8, 2, 15, 4, 8]
experiment(distribution_magnitude, n_participants, partition_types)

(0.56445, 0.1376, 0.07479999999999999, 0.7903, -0.018299999999999983, 0.7903)

In [11]:
state_p = {
    'control/untreated/bad': 0.32 / 2,
    'control/treated/bad': 0.32 / 2,
    'control/untreated/good': 0.04 / 2,
    'control/treated/good': 0.32 / 2,
    'treatment/untreated/bad': 0.02 / 2,
    'treatment/treated/bad': 0.17 / 2,
    'treatment/untreated/good': 0.67 / 2,
    'treatment/treated/good': 0.14 / 2
}

In [12]:
symbolic_solve(state_p)

(-0.17500000000000002, -0.07500000000000001)